<a href="https://colab.research.google.com/github/saibala2905/Prompt_To_NoSQL/blob/main/Prompt_To_NoSQL_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch

# Load the dataset
data_path = '/content/mongodb_queries_for_oee_expanded.csv'  # Update this path
data = pd.read_csv(data_path)

# Preprocess the data
def preprocess_data(data):
    data['input_text'] = "question: " + data['question']
    data['target_text'] = data['query']
    return data

data = preprocess_data(data)

# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1)

# Tokenization
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize(batch):
    tokenized_input = tokenizer(batch['input_text'], padding='max_length', truncation=True, max_length=512)
    tokenized_target = tokenizer(batch['target_text'], padding='max_length', truncation=True, max_length=512)

    return {
        'input_ids': tokenized_input.input_ids,
        'attention_mask': tokenized_input.attention_mask,
        'labels': tokenized_target.input_ids
    }

train_data = train_data.apply(tokenize, axis=1)
val_data = val_data.apply(tokenize, axis=1)

class T5Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        return {key: torch.tensor(val) for key, val in item.items()}

train_dataset = T5Dataset(train_data)
val_dataset = T5Dataset(val_data)

# Model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,  # Increase epochs as needed
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

# Save the model
model.save_pretrained('./t5_question_to_query')
tokenizer.save_pretrained('./t5_question_to_query')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

Step,Training Loss
10,15.203100
20,15.718000
30,14.956200
40,13.611000
50,12.845100
60,12.415800
70,11.564100
80,9.309400
90,7.596800
100,6.418700


('./t5_question_to_query/tokenizer_config.json',
 './t5_question_to_query/special_tokens_map.json',
 './t5_question_to_query/spiece.model',
 './t5_question_to_query/added_tokens.json')

In [25]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the trained model and tokenizer
model_path = '/content/t5_question_to_query'
tokenizer_path = '/content/t5_question_to_query'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

def generate_query(question, model, tokenizer):
    # Prepend 'question:' to the input question
    input_text = "question: " + question
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=True)
    # Generate the output (query)
    output_ids = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
    # Decode and return the generated text
    query = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return query

# Example usage
question = "IMM-01?"
query = generate_query(question, model, tokenizer)
print(query)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


db.getCollection('OEE_MOLD_OUT').find('MACHINE_NAME':'IMM-01').sort('_id':-1).project('MATERIAL_DESCRIPTION':1).limit(1)
